# Precipitation and Storms Data Analysis for CCEE Whitepaper

### Dataset and Library Imports

In [4]:
import pandas as pd
from utils import *

# extract the data sheets
school_data = pd.read_csv('../datasets/equity.csv')
storms_precip = pd.read_csv('../datasets/storms.csv')

### Data Cleaning

In [5]:
#simplified School_Data file
simplified_table = school_data.iloc[:, 1:26]
simplified_table.columns

Index(['CDS Code', 'County', 'County Superintendent (CCESSA) Region',
       'Geographical Lead Region', 'District Name', 'Abbreviation',
       'District Type', 'Grade Levels', 'Number of Schools',
       'High School Partner District if Elementary',
       'Total # of Jurisdictions Per School District',
       'Jurisdiction Name\n(list on separate line for each jurisdiction)',
       'Student Enrollment ', 'Size of District',
       'Total Teaching Positions (FTE)', '% Unduplicated ', '% FRM ',
       '% English Learners', '% Students Receiving Special Education Services',
       '% Students of Color', 'Expense of Education per ADA ',
       '# of Title 1 Schools', '% of Title 1 Schools',
       'Average Pollution Burden', 'Average Pollution Burden Score'],
      dtype='object')

In [6]:
wanted = storms_precip['Modeled Historical Baseline 1950-1999 (1975) Annual Avg Precip (in)']
# unwanted columns
not_wanted_avg = storms_precip.filter(regex='Avg').columns.values.tolist()
not_wanted_storm = storms_precip.filter(regex='Storm').columns.values.tolist()
not_wanted_diff = storms_precip.filter(regex='Dif').columns.values.tolist()

#splitting up the data into two different charts
#precipitation --> average might be more useful for grouping 
precip = storms_precip.drop(not_wanted_avg, axis = 1)
precip = precip.drop(not_wanted_storm, axis = 1)
precip = precip.rename({'County Name': 'County'}, axis='columns')
precip['Baseline'] = wanted

storm = storms_precip.drop(not_wanted_avg, axis = 1)
storm = storm.drop(not_wanted_diff, axis = 1)
storm = storm.rename({'County Name': 'County'}, axis='columns')
storm.columns

Index(['County', 'Climate Biome(s)', 'CDPH Climate Impact Regions',
       'Storm Threshold (in)',
       'Modeled Historical Baseline 1950-1999 (1975) Storm Events (per yr)',
       '2020-2029 (2025) Storm Events (per yr)',
       '2030-2039 (2035) Storm Events (per yr)',
       '2040-2049 (2045) Storm Events (per yr)',
       '2050-2059 (2055) Storm Events (per yr)'],
      dtype='object')

In [7]:
data = storms_precip.copy()
data = data.drop(not_wanted_avg, axis = 1)
data['Precip Baseline'] = wanted
data = data.drop('Storm Threshold (in)', axis = 1)
data

biomes = data['Climate Biome(s)']
data['biomes'] = biomes.str.split(',')
data= data.drop('Climate Biome(s)', axis =1)
data = data.explode('biomes')
data = data.reset_index().drop('index', axis = 1)
data['biomes'] = data['biomes'].str.strip()

## Vulnerability Score for Counties Based on Precipitation/Storm Events and School district information

***Below is the calulation and code for finding a vulnerability score. Used in the CCEE White paper as well as clasifying risks on the static maps. This information was caluclated based on the master School Csv file as well as the storm Precipitation file.***

Took both data sets of the Precipitation/ storm data and the County school information and combined them into a larger data set. Choose the equity feature from the County School information, in this case, % of Title One schools.
Generalized this information to find the percentage of Title One schools in each County rather than the district
Adding the total number of Title One Schools in the county / total number of schools in the County.

In [8]:
# DATA PREPED
data_1 = storms_precip.copy()
data_1 = data_1.drop(not_wanted_avg, axis = 1)
data_1['Precip Baseline'] = wanted
data_1 = data_1.drop('Storm Threshold (in)', axis = 1)

# Cleaning
simplified_table['# of Title 1 Schools'] = simplified_table['# of Title 1 Schools'].replace({',':'','-':''}, regex=True).astype(int)
simplified_table['Number of Schools'] = simplified_table['Number of Schools'].replace({',':'','-':''}, regex=True).astype(int)
simplified_table['Student Enrollment '] = simplified_table['Student Enrollment '].replace({',':''}, regex=True).astype(int)
simplified_table['% Unduplicated ' ] = simplified_table['% Unduplicated '].replace({'%': ''}, regex=True).astype(float)

# grouping by County
g_cout = simplified_table[['County','# of Title 1 Schools','Number of Schools']].groupby('County').sum()

# Calculate the % of Unduplicated students in the whole county
g_cout['% Unduplicated '] = simplified_table.groupby('County').apply(
    lambda x: (x['% Unduplicated '] * x['Student Enrollment ']).sum() / x['Student Enrollment '].sum()
)
g_cout = g_cout.reset_index()

data_1 = data_1.rename({'County Name': 'County'}, axis='columns')
data_1 = data_1.merge(g_cout, on = 'County')

# get the percentage of title 1 schools in the whole county
data_1['title one'] = (g_cout['# of Title 1 Schools']/g_cout['Number of Schools'])*100

/var/folders/dv/l2nd8t1935qg0dz8vg__3m980000gn/T/ipykernel_72660/1206397741.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  g_cout['% Unduplicated '] = simplified_table.groupby('County').apply(


## Precipitation

Steps of analysis: 
1. Calculated the % change of the precipitation per decade (increase or decrease) (Average inches of rain in that decade / historical baseline 1950-1999 (inches of rain) ) x 100
2. Combined the calculated percentages, % of title one, % precipitation change, % of Unduplicated Students When combining these percentages, a weight of 0.33 had to be multiplied by each feature. This would calculate the weighted average between all features allowing a balance within the score used.
3. Based on those scores the score is at the 75th percentile and the score at the 50th percentile to apply a filter for the necessary data points *Solely based on the range of values*
4. Using those values a filer was applied to find all the counties above the 75th percentile and between the 50th and 75th percentile for the next 25% vulnerable.
5. Once all points are filtered, the corresponding values are summed up to get the number on the table, [ number of schools, student enrollment, number of districts, number of counties within that percent].




In [9]:
analyze_data(data_1, simplified_table,
             '2020-2029 (2025) Difference from Baseline (in)', 
             'Precip Baseline', 
             'Precipitation')

Number of counties in the top 25%: 18
Number of Schools         4920
Student Enrollment     2897906
Grade Levels               405
dtype: int64
Number of counties in the next 25%: 20
Number of Schools        1799
Student Enrollment     927046
Grade Levels              226
dtype: int64
Num schools:  6719
Students:  3824952
Num districts:  631


## Storm Events

Steps of analysis: 
1. Calculated the % change of the storm events per decade (increase or decrease)
(Average # of storm events in that decade / historical baseline 1950-1999 (storm events)) x 100
2. Combined the calculated percentages, % of title one, % precipitation change, % of Unduplicated Students When combining these percentages, a weight of 0.33 had to be multiplied by each feature. This would calculate the weighted average between all features allowing a balance within the score used.
3. Based on those scores the score is at the 75th percentile and the score at the 50th percentile to apply a filter for the necessary data points *Solely based on the range of values*
4. Using those values a filer was applied to find all the counties above the 75th percentile and between the 50th and 75th percentile for the next 25% vulnerable.
5. Once all points are filtered, the corresponding values are summed up to get the number on the table, [ number of schools, student enrollment, number of districts, number of counties within that percent].

In [11]:
analyze_data(data_1, simplified_table,
             '2020-2029 (2025) Storm Events (per yr)', 
             'Modeled Historical Baseline 1950-1999 (1975) Storm Events (per yr)', 
             'Storms')

Number of counties in the top 25%: 10
Number of Schools        1143
Student Enrollment     614456
Grade Levels              169
dtype: int64
Number of counties in the next 25%: 12
Number of Schools         4094
Student Enrollment     2447074
Grade Levels               272
dtype: int64
Num schools:  5237
Students:  3061530
Num districts:  441
